In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch import nn, optim
from torchvision.models import mobilenet_v2
from PIL import Image
from sklearn.metrics import accuracy_score


# Define transformations for input images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

c:\Users\Al Amin\anaconda3\envs\deep_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define dataset path
dataset_path = 'Grp_three_dataset/data_train_test/train'

# Load the dataset
dataset = datasets.ImageFolder(dataset_path, transform=transform)

# Split the dataset into training and testing sets (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the model
model = mobilenet_v2(pretrained=True)
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_features, len(dataset.classes))
)

c:\Users\Al Amin\anaconda3\envs\deep_learning\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Al Amin\anaconda3\envs\deep_learning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
model.train()
for epoch in range(5):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/5], Loss: {epoch_loss:.4f}')

# Evaluate the model
model.eval()
true_labels = []
predicted_labels = []
for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    true_labels.extend(labels.tolist())
    predicted_labels.extend(predicted.tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'train_model.pth')


model.load_state_dict(torch.load('train_model.pth'))
model.eval()

Epoch [1/5], Loss: 1.3530
Epoch [2/5], Loss: 0.8006
Epoch [3/5], Loss: 0.6275
Epoch [4/5], Loss: 0.4803
Epoch [5/5], Loss: 0.4113
Accuracy: 0.8008


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [4]:
# Perform inference on a sample image
sample_image_path = 'Grp_three_dataset/data_train_test/test/Akara and Eko/01-Akara-Black-Eyed-Pea-Fritters-from-fryer-second-shot.jpg'
sample_image = Image.open(sample_image_path)
sample_input = transform(sample_image).unsqueeze(0)
with torch.no_grad():
    output = model(sample_input)
predicted_class = torch.argmax(output, dim=1).item()

# Print predicted class label
print(f'Predicted class: {dataset.classes[predicted_class]}')

Predicted class: Akara and Eko


In [5]:
import torch
from torchvision.models import mobilenet_v2
from torch import nn

# Assuming you have saved your trained model as 'mobilenet_v2_model.pth'
model_path = 'train_model.pth'

# Load the model
loaded_model = mobilenet_v2(pretrained=False)  # Initialize the model architecture
num_features = loaded_model.classifier[1].in_features
loaded_model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_features, len(dataset.classes))
)
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.eval()  # Set the model to evaluation mode

# Perform inference on a sample image
sample_image_path = 'Grp_three_dataset/data_train_test/test/Akara and Eko/01-Akara-Black-Eyed-Pea-Fritters-from-fryer-second-shot.jpg'
sample_image = Image.open(sample_image_path)
sample_input = transform(sample_image).unsqueeze(0)
with torch.no_grad():
    output = loaded_model(sample_input)
predicted_class = torch.argmax(output, dim=1).item()

# Print predicted class label
print(f'Predicted class: {dataset.classes[predicted_class]}')


c:\Users\Al Amin\anaconda3\envs\deep_learning\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Al Amin\anaconda3\envs\deep_learning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Predicted class: Akara and Eko


In [6]:
predicted_class

1

In [7]:
dataset.classes

['Abacha and Ugba(african salad)',
 'Akara and Eko',
 'Amala and Gbegiri- Ewedu',
 'Asaro',
 'Boli(bole)',
 'Chin Chin',
 'Egusi Soup',
 'Ewa-Agoyin',
 'Fried Plantains (dodo)',
 'Jollof Rice',
 'Meat-pie',
 'Moin-Moin',
 'Nkwobi',
 'Okro Soup',
 'Pepper Soup',
 'Puff-Puff',
 'Suya',
 'Vegetable Soup']